<a href="https://colab.research.google.com/github/froge159/belief-project-sef/blob/main/phase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import random
import pandas as pd
# install bitsandbytes and restart

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dreaddit = pd.read_csv('/content/drive/MyDrive/SEF/data/dreaddit.csv')

In [ ]:
model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, padding=True, truncation=True, model_max_length=512)
tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)
model.eval()

In [ ]:
num_layers = model.config.num_hidden_layers
num_heads = model.config.num_attention_heads
head_dim = model.config.hidden_size
device = "cuda" if torch.cuda.is_available() else "cpu"

# Extract Mean-Pooled Activations

In [ ]:
def extract_mean_pooled_hidden(prompts, batch_size=8):
  all_pooled = []
  for i in range(0, len(prompts), batch_size):
    batch = prompts[i:i+batch_size]
    inputs = tokenizer(
        batch,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
      outputs = model(**inputs, output_hidden_states=True)

    hidden_states = outputs.hidden_states[1:]
    attention_mask = inputs["attention_mask"] # [bs, seq_len]
    mask = attention_mask.unsqueeze(-1)
    lengths = attention_mask.sum(dim=1).unsqueeze(-1)

    pooled_layers = []
    for hs in hidden_states:
        # hs: [bs, seq_len, hidden_dim]
        masked_sum = (hs * mask).sum(dim=1)     # [bs, hidden_dim]
        mean_pooled = masked_sum / lengths      # [bs, hidden_dim]
        pooled_layers.append(mean_pooled)

    # Stack layers: [bs, num_layers, hidden_dim]
    batch_pooled = torch.stack(pooled_layers, dim=1)
    all_pooled.append(batch_pooled.cpu())

  return torch.cat(all_pooled, dim=0)

In [ ]:
mean_pooled = extract_mean_pooled_hidden(dreaddit['text'].tolist())

# Train Linear Probe

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

def train_linear_probe(mean_pooled, labels, n_splits=5, seed=42):
    """
    Trains a linear probe for each layer using cross-validation.
    Args:
        mean_pooled: torch.Tensor [n, num_layers, hidden_dim]
        labels: array-like, shape [n]
        n_splits: int, number of CV folds
        seed: int, random seed
    Returns:
        pd.DataFrame with metrics for each layer
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    X_all = mean_pooled.detach().cpu().numpy()
    y_all = np.array(labels, dtype=float)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    results = []
    for layer_idx in range(X_all.shape[1]):
        X_layer = X_all[:, layer_idx, :]
        y_true_all = []
        y_pred_all = []
        for train_idx, test_idx in kf.split(X_layer):
            X_train, X_test = X_layer[train_idx], X_layer[test_idx]
            y_train, y_test = y_all[train_idx], y_all[test_idx]
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_true_all.append(y_test)
            y_pred_all.append(y_pred)
        y_true_all = np.concatenate(y_true_all)
        y_pred_all = np.concatenate(y_pred_all)
        if np.std(y_pred_all) == 0 or np.std(y_true_all) == 0:
            pearson = np.nan
        else:
            pearson = np.corrcoef(y_true_all, y_pred_all)[0, 1]
        r2 = r2_score(y_true_all, y_pred_all)
        mae = mean_absolute_error(y_true_all, y_pred_all)
        results.append({
            "layer": layer_idx,
            "pearson": pearson,
            "r2": r2,
            "mae": mae,
        })
    return pd.DataFrame(results)

# Call the function and display results
test_results = train_linear_probe(mean_pooled, dreaddit["belief_score"].values)
test_results

# Validation Analyses

### Probe Validity Checks

In [ ]:
# Shuffle labels and run probe
shuffled_labels = dreaddit["belief_score"].sample(frac=1, random_state=42).values
shuffled_results = train_linear_probe(mean_pooled, shuffled_labels)

# Compare early vs late layers for original and shuffled
num_layers = mean_pooled.shape[1]
early_layer = 0
late_layer = num_layers - 1

print("Original probe metrics (early vs late layer):")
print(test_results.loc[[early_layer, late_layer], ["layer", "pearson", "r2", "mae"]])

print("\nShuffled probe metrics (early vs late layer):")
print(shuffled_results.loc[[early_layer, late_layer], ["layer", "pearson", "r2", "mae"]])

### Visualization Summary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Layer-wise R^2 for original vs shuffled labels
plt.figure(figsize=(8, 4))
plt.plot(test_results["layer"], test_results["r2"], marker="o", label="Original")
plt.plot(shuffled_results["layer"], shuffled_results["r2"], marker="o", label="Shuffled")
plt.xlabel("Layer index")
plt.ylabel("R^2")
plt.title("Layer-wise Regression Performance")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Pick best layer by R^2 and compute predictions
best_layer = int(test_results.sort_values("r2", ascending=False).iloc[0]["layer"])
X_best = mean_pooled[:, best_layer, :].detach().cpu().numpy()
y_true = np.array(dreaddit["belief_score"].values, dtype=float)
best_model = LinearRegression()
best_model.fit(X_best, y_true)
y_pred = best_model.predict(X_best)

# Predicted vs true scatter with regression line
plt.figure(figsize=(5, 5))
plt.scatter(y_true, y_pred, alpha=0.5)
m, b = np.polyfit(y_true, y_pred, 1)
xs = np.linspace(y_true.min(), y_true.max(), 100)
plt.plot(xs, m * xs + b, color="red", label="Fit line")
plt.xlabel("True belief strength")
plt.ylabel("Predicted belief strength")
plt.title(f"Predicted vs True (Layer {best_layer})")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Violin plot of predicted belief strength by group
group_labels = np.array(dreaddit["category"].values)
pred_depressive = y_pred[group_labels == "DEPRESSIVE"]
pred_control = y_pred[group_labels == "CONTROL"]

plt.figure(figsize=(6, 4))
plt.violinplot([pred_depressive, pred_control], showmeans=True)
plt.xticks([1, 2], ["DEPRESSIVE", "CONTROL"])
plt.ylabel("Predicted belief strength")
plt.title("Predicted Belief Strength by Group")
plt.grid(axis="y", alpha=0.3)
plt.show()

### Group Level Comparison

In [ ]:
# freeze the trained probe
# compute predicted belief strength for each text and calculate depressed vs control group means. 
# then compare distributions, compute effect size, and run a simple statistical test. 

from scipy.stats import ttest_ind

def group_level_probe(mean_pooled, labels, group_labels, layer_idx=0):
    """
    Use trained probe on a given layer to predict belief strength, compare group means, effect size, and t-test.
    Args:
        mean_pooled: torch.Tensor [n, num_layers, hidden_dim]
        labels: array-like, shape [n]
        group_labels: array-like, shape [n] (e.g., 'DEPRESSIVE' or 'CONTROL')
        layer_idx: int, layer to use for probe
    Returns:
        dict with group means, effect size, t-test results
    """
    # Train probe on all data for selected layer
    X = mean_pooled[:, layer_idx, :].detach().cpu().numpy()
    y = np.array(labels, dtype=float)
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    # Split by group
    group1 = y_pred[np.array(group_labels) == 'DEPRESSIVE']
    group2 = y_pred[np.array(group_labels) == 'CONTROL']
    mean1 = np.mean(group1)
    mean2 = np.mean(group2)
    std1 = np.std(group1)
    std2 = np.std(group2)
    # Cohen's d
    pooled_std = np.sqrt(((len(group1)-1)*std1**2 + (len(group2)-1)*std2**2) / (len(group1)+len(group2)-2))
    cohens_d = (mean1 - mean2) / pooled_std
    # t-test
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
    return {
        'DEPRESSIVE_mean': mean1,
        'CONTROL_mean': mean2,
        "cohens_d": cohens_d,
        "t_stat": t_stat,
        "p_val": p_val,
    }

# Run group-level comparison for the last layer (best probe)
group_stats = group_level_probe(mean_pooled, dreaddit["belief_score"].values, dreaddit["category"].values, layer_idx=mean_pooled.shape[1]-1)
print("Group-level comparison (last layer):")
for k, v in group_stats.items():
    print(f"{k}: {v}")

### Verify Similarity of Token-Length Distribution

In [ ]:
# addresses maybe longer posts look more hopeless type critiques
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Compute token lengths for each text
text_tokens = dreaddit['text'].apply(lambda x: len(tokenizer.tokenize(x)))
dreaddit['token_length'] = text_tokens

depressive_lengths = dreaddit.loc[dreaddit['category'] == 'DEPRESSIVE', 'token_length']
control_lengths = dreaddit.loc[dreaddit['category'] == 'CONTROL', 'token_length']

# Statistical test (Mann-Whitney U, nonparametric)
stat, p = mannwhitneyu(depressive_lengths, control_lengths, alternative='two-sided')
print(f"Mann-Whitney U test: statistic={stat}, p-value={p}")

# Plot distributions
plt.figure(figsize=(8,5))
plt.hist(depressive_lengths, bins=30, alpha=0.6, label='DEPRESSIVE')
plt.hist(control_lengths, bins=30, alpha=0.6, label='CONTROL')
plt.xlabel('Token Length')
plt.ylabel('Count')
plt.title('Token Length Distribution by Category')
plt.legend()
plt.show()